In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

%tensorflow_version 2.x
!pip install --pre tensorflow-gpu==2.0.0a0
!pip install -q tf-nightly-2.0-preview
!pip install --upgrade --force-reinstall tb-nightly

In [0]:
from packaging import version

import tensorflow as tf
print("TensorFlow version: ", tf.__version__)
from tensorflow.keras import Model, datasets
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from datetime import datetime as ddt
                                      
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

In [0]:
!rm -rf ./logs/ 
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [0]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).shuffle(10000)
train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y))
train_dataset = train_dataset.map(lambda x, y: (tf.image.random_flip_left_right(x), y))

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).shuffle(10000)
test_dataset = test_dataset.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y))

In [0]:
class ThreeConvs(Model):
  def __init__(self):
    super(ThreeConvs, self).__init__()
    self.conv1 = Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3))
    self.pool1 = MaxPooling2D((2, 2))
    self.conv2 = Conv2D(64, (3, 3), activation='relu')
    self.pool2 = MaxPooling2D((2, 2))
    self.conv3 = Conv2D(128, (3, 3), activation='relu')
    self.flat = Flatten()
    self.fc1 = Dense(512, activation='relu')
    self.dropout = Dropout(0.5)
    self.fc2 = Dense(10, activation='softmax')


  def call(self, x, training = False):
    x = self.conv1(x)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = self.conv3(x)
    x = self.flat(x)
    x = self.fc1(x)
    if training:
        x = self.dropout(x)
    return self.fc2(x)

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [0]:
current_time = ddt.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/lab4/' + current_time + '/train'
test_log_dir = 'logs/lab4/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
@tf.function
def train_step(image, label):
  with tf.GradientTape() as tape:
    predictions = model(image)
    loss = loss_object(label, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(label, predictions)

In [0]:
@tf.function
def test_step(image, label):
  predictions = model(image)
  t_loss = loss_object(label, predictions)

  test_loss(t_loss)
  test_accuracy(label, predictions)

In [0]:
model = ThreeConvs() # reset model

ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

EPOCHS = 8
template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
template_step = 'Step {}, Loss: {}, Accuracy: {}'

ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  step_num = int(ckpt.step)
  print(tf.train.list_variables('./tf_ckpts'))
  print("Restored from {}".format(manager.latest_checkpoint))
else:
  step_num = 0
  print("Initializing from scratch.")

%tensorboard --logdir logs/lab4

for epoch in range(EPOCHS):
  i = 0
  for image, label in train_dataset:
      train_step(image, label)
      ckpt.step.assign_add(1)
      with train_summary_writer.as_default():
          tf.summary.scalar('loss', train_loss.result(), step=step_num)
          tf.summary.scalar('accuracy', train_accuracy.result(), step=step_num)
      if i % 100 == 0:
          print(template_step.format(i+1, train_loss.result(), train_accuracy.result()*100))
      step_num += 1
      i += 1

  for test_image, test_label in test_dataset:
    test_step(test_image, test_label)
  with test_summary_writer.as_default():
      tf.summary.scalar('loss', test_loss.result(), step=step_num)
      tf.summary.scalar('accuracy', test_accuracy.result(), step=step_num)

  ckpt.mapped = {'step_num': step_num}
  save_path = manager.save()
  print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
  
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))
  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

In [0]:
!ls ./tf_ckpts